In [9]:
# Defining the library
import pandas as pd
import numpy as np
from FREDMD_Tools import*

In [10]:
# Declaring dates staritng from 2005/1/1 to 2011/3/1
backtest_dates = pd.date_range(start='2005-1-1', end='2011-3-1',freq='M').to_period('M')


In [11]:
# Defining a function to load the data
#This will read, drop, set the column to 'sasdate' as index
def load_data(filename):
    df = pd.read_csv(filename,header=0)
    df = df.drop(0)
    df = df.set_index('sasdate')
    df.index = pd.PeriodIndex(df.index, freq = 'M')
    return df
    

In [12]:
FRED_DATA = load_data('2023-09.csv')

In [13]:
# Creating dataframe model to store outcomes
models = pd.DataFrame(index=FRED_DATA.columns)
naive = pd.DataFrame(index=backtest_dates, columns=FRED_DATA.columns)

In [14]:
horizons = [3,6,9,12]

In [15]:
for h in horizons:
    # Establish dataframe to store errors
    globals()[f'horizon_{h}'] = pd.DataFrame(index=backtest_dates,columns=FRED_DATA.columns)
    # Establish dataframe to store naive errors
    globals()[f'naive_{h}'] = pd.DataFrame(index=backtest_dates,columns=FRED_DATA.columns)

In [16]:
# looping by backtest date
for date in backtest_dates:
    # Looping each and every series
    for series in FRED_DATA.columns:
        # Choosing the appropriate data to backtest date
        srs = select_continuous(FRED_DATA[series][:date])
        model_1,frcst = fit_models(srs, h=12)
        models.loc[series,date] = model_1
        naive.loc[date,series] = srs.iloc[-1]
        
        # Looping the horizons
        for h in horizons:
            globals()[f'horizon_{h}'].loc[date,series] = (FRED_DATA[series][date+1:date+12] - frcst)[h-1]
            globals()[f'naive_{h}'].loc[date,series] = FRED_DATA[series][date+h] - naive.loc[date,series]
            
       
       

KeyboardInterrupt: 

In [17]:
# Creating dataframes for the R-squared values
for h in horizons:
    globals()[f'R2_{h}M'] = pd.DataFrame(index = FRED_DATA.columns,columns=['R2'])

# loop series
for series in FRED_DATA.columns:
    for h in horizons:
        # Calculating the R square
        residual = globals()[f'horizon_{h}'].loc[:,series].pow(2).sum()
        totals = globals()[f'naive_{h}'].loc[:,series].pow(2).sum()
        R2 = 1 - residual/totals
        # Storing the R-squared values
        globals()[f'R2_{h}M'].at[series, 'R2'] = R2

C:\Users\ACER\AppData\Local\Temp\ipykernel_13720\3211974167.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  R2 = 1 - residual/totals


ZeroDivisionError: division by zero

In [18]:
# Constructing a data with identifiers
frcst_residual = pd.DataFrame(index = FRED_DATA.columns, columns=['model','R2'])

In [19]:
# Saving the model and the R2 values
frcst_residual['model'] = models.iloc[:, -1]
frcst_residual['R2'] = R2_3M

In [21]:
# Saving the data into CSV file
frcst_residual.to_csv('fcast_rese.csv')


In [22]:
frcst_residual

,model,R2
RPI,AA,-0.074117
W875RX1,AAd,-2.616867
DPCERA3M086SBEA,AA,0.965638
CMRMTSPLx,AAd,0.999404
RETAILx,AA,0.889061
...,...,...
UMCSENTx,NaN,NaN
DTCOLNVHFNM,NaN,NaN
DTCTHFNM,NaN,NaN
INVEST,NaN,NaN
